# Locating scans for the LongitudinalDiffusion Project on Flywheel
Get Flywheel **acqid** and **filename** for `anat`, `dwi`, and `fmap` files from all subjects/sessions in the following projects:
- 22Q_812481
- 22q_Midline_834246
- EFR01
- EONSX_810366
- Evolution_833922
- GRMPY_822831
- PNC_CS_810336
- PNC_LG_810336
- SYRP_818621

**Project:** LongitudinalDiffusion <br>
**Author:** Katja Zoner <br>
**Date:** 02/24/2022 <br>

In [102]:
from tqdm import tqdm
from datetime import datetime
import numpy as np
import pandas as pd
import flywheel

In [25]:
# Get client
fw = flywheel.Client()
assert fw, "Your Flywheel CLI credentials aren't set!"

In [180]:
PROJECT_LIST = [
    "22Q_812481",
    "22q_Midline_834246",
    "EFR01",
    "EONSX_810366",
    "Evolution_833922",
    "GRMPY_822831",
    "PNC_CS_810336",
    "PNC_LG_810336",
    "SYRP_818621"
]

BIDSIFIED_LIST = [
    "22Q_812481",
    "EFR01",
    "GRMPY_822831",
    "PNC_CS_810336",
    "PNC_LG_810336",
    "SYRP_818621"
]

In [26]:
def get_project(fw, label):
    '''
    Helper function to retrieve Flywheel project given project name.
    '''
    project = fw.projects.find_first('label="{}"'.format(label))
    assert project, f"Project '{label}' not found on Flywheel!"  
    return project

In [163]:
def get_from_bidsified(project, info):
    '''
    Loop through all sessions in a BIDSIFIED project to gather dict of 
    all niftis and associated files under the anat, dwi, and fmap folders.
    '''
    # Loop through subjects in project
    for sub in tqdm(project.subjects(), desc=f"Subjects processed", unit="subject", position=0):
        
        # Loop through sessions in subject
        for ses in sub.sessions():
            ses = ses.reload()
            
            # Loop through acquisitions in session
            for acq in ses.acquisitions():
                acq = acq.reload()

                # Loop through files in acquisition
                for f in acq.files:
                    
                    # If file is one of nifti/bval/bvec, try to get bids folder data.
                    if f.type in ['nifti', 'bval', 'bvec']:  
                        
                        try:
                            bids_folder = f.info['BIDS']['Folder']
                            # Add file metadata to info dict if file falls under one of anat, dwi, fmap BIDS dirs.
                            if bids_folder in ['anat', 'dwi', 'fmap']:
                                modality = f.info['BIDS']['Filename'].split(".")[0].split("_")[-1]
                                info[f.id] =  [project.label, sub.label, ses.label, acq.id, bids_folder, modality, f.name]

                        # KeyError if file info doesn't have BIDS entry; TypeError if it has BIDS that isn't a dict 
                        except (KeyError, TypeError) as e:
                        # except KeyError:
                            pass


            

## 1. Build dictionary of scan data for each anat/dwi/fmap file in the bidsified projects

In [164]:
# Create info dict with entries for each scan.
info = {}

In [178]:
done = []
# Loop through each BIDSIFIED project and locate scans. 
# Note: this takes a while, esp for projects with many subjects!
for project_label in BIDSIFIED_LIST:
    project = get_project(fw, project_label,)
    get_from_bidsified(project, info)
    done.append(project_label)
    print(f"Finished locating scans from project {project_label}.")

Subjects processed: 100%|██████████| 1601/1601 [3:00:16<00:00,  6.76s/subject]  


In [181]:
len(info)

28768

## 2. Convert dict to dataframe and clean data

In [264]:
def printCounts(df):
    print(f"Subject count: {len(df.bblid.unique())}")
    print(f"Sessions count: {len(df.sesid.unique())}")

In [352]:
# Convert info dict to pandas dataframe, rename columns, remove fileid col
df = pd.DataFrame.from_dict(info, orient='index').reset_index()

# Rename columns
df.columns=['fileid', 'project', 'bblid', 'sesid', 'acqid', 'folder', 'modality', 'filename']

# Remove fileid column
# del df['fileid']

# Convert bblid and sesid to str
df['bblid'] = df.bblid.astype(str)
df['sesid'] = df.sesid.astype(str)

# WHAT TO DO WITH NON-DIGIT SESSION LABELS --> for now, removing these rows
df = df[df.sesid.apply(lambda x: x.isnumeric())]

# Strip additional leading zero from sesid (22Q project issue)
df.loc[df.project == '22Q_812481', 'sesid'] = df.loc[df.project == '22Q_812481', 'sesid'].apply(lambda x: x.lstrip('0'))

# Add leading zeros to bblid and sesid so ids are 6 and 5 digits long respectively
df['bblid'] = df.bblid.apply(lambda x: x.zfill(6))
df['sesid'] = df.sesid.astype(str).apply(lambda x: x.zfill(5))

df

,fileid,project,bblid,sesid,acqid,folder,modality,filename
0,f8788782-8ae1-4e3e-bcea-611dd109614b,22Q_812481,016533,09422,5c8fb15df546b6002ebda7f0,fmap,phasediff,B0map_onesizefitsall_v4_7_ph.nii.gz
1,ccbe964f-fce0-4b5a-bd8c-e15b3aa23546,22Q_812481,016533,09422,5c8fb15df546b6002ebda7f1,fmap,magnitude2,B0map_onesizefitsall_v4_6_e2.nii.gz
2,4bc396b8-e4f6-411f-a5e5-02ee0f9268d5,22Q_812481,016533,09422,5c8fb15df546b6002ebda7f1,fmap,magnitude1,B0map_onesizefitsall_v4_6_e1.nii.gz
3,09dfc861-630b-4b67-9933-6a7eed5a0f6d,22Q_812481,016533,09422,5c8fb15df546b6002fbdafc9,anat,T1w,MPRAGE_TI1100_ipat2_2.nii.gz
4,3cf61378-bbe8-49c2-bae4-13e8f1f486a2,22Q_812481,016533,09422,5c8fb15df546b6002bbd9d23,dwi,dwi,DTI_2x32_35_10.nii.gz
...,...,...,...,...,...,...,...,...
28763,2ba18d00-06e0-42bb-b9de-8ca55643952a,PNC_CS_810336,080010,02894,5d94bbf7a54d350042b4b970,fmap,magnitude2,B0map_onesizefitsall_v2_10_e2.nii.gz
28764,4096e82d-59d8-4dfe-8abb-d95dae4cafb3,PNC_CS_810336,080010,02894,5d94bbf7a54d350042b4b970,fmap,magnitude1,B0map_onesizefitsall_v2_10_e1.nii.gz
28765,411c7252-3bbb-4088-b861-e25074fcc03b,PNC_CS_810336,080010,02894,5d94bbf8a54d350042b4b971,fmap,phase1,B0map_onesizefitsall_v2_11_e1_ph.nii.gz
28766,be35110b-cfc8-418e-9d2f-460fafa8792c,PNC_CS_810336,080010,02894,5d94bbf8a54d350042b4b971,fmap,phase2,B0map_onesizefitsall_v2_11_e2_ph.nii.gz


In [353]:
# Print count of subjects with one of anat/dwi/fmap scans located by this script
printCounts(df)

Subject count: 1922
Sessions count: 2986


## 3. QC Dataframe --> check for 2+ sessions, and presence of T1w, dwi, fmap modalities

In [243]:
def get_t1w(session):
    return session[session.modality == "T1w"]

def get_dwi(session):
    return session[session.folder == "dwi"]

def get_fmap(session):
    return session[session.folder == "fmap"]

In [261]:
def get_required_files_from_valid_sessions(df):
    '''
    Filter to only include sessions that contain a T1w image, and scans under both fmap and dwi bids folders.
    Note: Could add better QC checks for project level dwi/fmap requirements, but this was easiest for a short deadline.
    '''
    required = pd.DataFrame(columns=df.columns)

    for sesid in df.sesid.unique():
        session_df = df[df.sesid == sesid]
        assert len(session_df.project.unique()) == 1, f"Session label {sesid} was found under multiple projects!"
        assert len(session_df.bblid.unique()) == 1, f"Session label {sesid} was found under multiple subjects!."
        
        t1w = get_t1w(session_df)
        dwi = get_dwi(session_df)
        fmap = get_fmap(session_df)

        if not t1w.empty and not dwi.empty and not fmap.empty:
            required = pd.concat([required, t1w, dwi,fmap], ignore_index=True)
        # else:
        #     print(f"Session {sesid} does not have all required scans")
    return required
    

In [346]:
def filter_by_session_count(df, thresh):
    '''
    Filter dataframe to only include subjects who have <threshold> or more scans.
    '''

    df = df.sort_values(["bblid","sesid"])

    # Get session counts for each subject (bblid).
    bblid_counts = df.groupby(["bblid", "sesid"]).size().reset_index().groupby(["bblid"]).size().reset_index()
    bblid_counts.columns = ["bblid", "session_count"]
    bblid_counts.sort_values("session_count")

    # Get list of bblids that should be included
    include_bblids = bblid_counts.bblid[bblid_counts["session_count"]>=thresh]

    # Filter dataframe to only include subjects that meet session requirements
    df = df[df["bblid"].isin(include_bblids)].copy()

    # Update ntimepoints column to indicate number of sessions in ExtraLong 2021 for each subject.
    for bblid in df.bblid.unique():

        # Get subject's number of timepoints in ExtraLong 2021
        num_tps = bblid_counts[bblid_counts.bblid == bblid].session_count.item()

        # Update ntimeponts in ExtraLong dataframe
        df.loc[df.bblid==bblid, 'ntimepoints'] = num_tps

        tp_count = 1
        for sesid in df[df.bblid == bblid].sesid.unique():
            df.loc[(df.bblid == bblid) & (df.sesid == sesid), 'timepoint' ] = tp_count
            tp_count += 1

    return df.sort_values(["bblid","sesid"])

In [354]:
# 1. Filter to only include sessions with T1w, dwi, and fmaps
upload_df = get_required_files_from_valid_sessions(df)
upload_df


,fileid,project,bblid,sesid,acqid,folder,modality,filename
0,09dfc861-630b-4b67-9933-6a7eed5a0f6d,22Q_812481,016533,09422,5c8fb15df546b6002fbdafc9,anat,T1w,MPRAGE_TI1100_ipat2_2.nii.gz
1,3cf61378-bbe8-49c2-bae4-13e8f1f486a2,22Q_812481,016533,09422,5c8fb15df546b6002bbd9d23,dwi,dwi,DTI_2x32_35_10.nii.gz
2,e5558eb2-cc09-49b0-b1a2-48b9f9250d7f,22Q_812481,016533,09422,5c8fb15df546b6002bbd9d23,dwi,dwi,DTI_2x32_35_10.bvec
3,55c51ef0-6b81-4ae2-befb-10dceb8a74e5,22Q_812481,016533,09422,5c8fb15df546b6002bbd9d23,dwi,dwi,DTI_2x32_35_10.bval
4,a7c7e37d-8a8f-44c7-b052-8548bff6cfd1,22Q_812481,016533,09422,5c8fb15df546b6002ebda7f2,dwi,dwi,DTI_2x32_36_11.nii.gz
...,...,...,...,...,...,...,...,...
25859,32876b16-ff4c-46a7-ae82-bc4ed76e3fcb,PNC_CS_810336,080010,02894,5d94bbf8a54d35003eb2d010,dwi,dwi,DTI_2x32_36_9.bval
25860,2ba18d00-06e0-42bb-b9de-8ca55643952a,PNC_CS_810336,080010,02894,5d94bbf7a54d350042b4b970,fmap,magnitude2,B0map_onesizefitsall_v2_10_e2.nii.gz
25861,4096e82d-59d8-4dfe-8abb-d95dae4cafb3,PNC_CS_810336,080010,02894,5d94bbf7a54d350042b4b970,fmap,magnitude1,B0map_onesizefitsall_v2_10_e1.nii.gz
25862,411c7252-3bbb-4088-b861-e25074fcc03b,PNC_CS_810336,080010,02894,5d94bbf8a54d350042b4b971,fmap,phase1,B0map_onesizefitsall_v2_11_e1_ph.nii.gz


In [355]:
print(f"After excluding sessions without T1w, dwi, and fmp, we have:")
printCounts(upload_df)

After excluding sessions without T1w, dwi, and fmp, we have:
Subject count: 1687
Sessions count: 2599


In [356]:
# # 2. Final filter on resulting dataframe to only include subjects with 2+ valid sessions.
upload_df = filter_by_session_count(upload_df, 2)
upload_df.ntimepoints = upload_df.ntimepoints.astype(int)
upload_df.timepoint = upload_df.timepoint.astype(int)
upload_df

,fileid,project,bblid,sesid,acqid,folder,modality,filename,ntimepoints,timepoint
267,265b5c62-ce67-4726-94c5-ace6bc842391,22Q_812481,015305,08932,5c8fb164f546b6002cbd9f9a,anat,T1w,MPRAGE_TI1100_ipat2_2.nii.gz,2,1
268,35346e48-ef41-4051-a701-6926ad04f709,22Q_812481,015305,08932,5c8fb164f546b6002fbdafeb,dwi,dwi,DTI_2x32_36_11.nii.gz,2,1
269,4a8dd468-3aae-4b66-a3a9-50cc35edfaf8,22Q_812481,015305,08932,5c8fb164f546b6002fbdafeb,dwi,dwi,DTI_2x32_36_11.bvec,2,1
270,0576830c-8bfe-42c3-8413-bf5c077b4860,22Q_812481,015305,08932,5c8fb164f546b6002fbdafeb,dwi,dwi,DTI_2x32_36_11.bval,2,1
271,e7357eb0-48ce-480d-9377-690fc687b962,22Q_812481,015305,08932,5c8fb164f546b6002fbdafec,dwi,dwi,DTI_2x32_35_10.nii.gz,2,1
...,...,...,...,...,...,...,...,...,...,...
3795,2540c846-df7a-47c4-ba9f-039cb95a77a2,PNC_LG_810336,139272,10040,5c76ef2dba2580003839f9fe,dwi,dwi,DTI_2x32_35_13.bvec,2,2
3796,a8036e95-7521-4472-90cf-b0df2bd3e689,PNC_LG_810336,139272,10040,5c76ef2dba2580003839f9fe,dwi,dwi,DTI_2x32_35_13.bval,2,2
3797,d3ff6180-1138-454c-a950-a2d2364125c3,PNC_LG_810336,139272,10040,5c76ef2dba2580002938f366,fmap,phasediff,B0map_v4_9_ph.nii.gz,2,2
3798,5ac4cb6e-6777-456b-bf39-72a5c6f494ce,PNC_LG_810336,139272,10040,5c76ef2dba2580003839f9f7,fmap,magnitude2,B0map_v4_8_e2.nii.gz,2,2


In [357]:
print(f"After excluding subjects with less than 2 sessions we have:")
printCounts(upload_df)

After excluding subjects with less than 2 sessions we have:
Subject count: 601
Sessions count: 1513


In [358]:
# Export dataframe to csv
filename = f"scans_for_download_upload_{datetime.now().isoformat()}.csv"
upload_df.to_csv("../../data/organize/" + filename, index=False)